# Population connectivity

### R Dependancies

* vcfR v1.13.0
* Rtsne v0.16
* adegenet v2.1.8
* dartR v2.7.2
* tidyverse v1.3.2
* reshape2 v1.4.4

In [ ]:
# Install packages if needed

#install.packages("vcfR");
#install.packages("Rtsne");
#install.packages("adegenet");
#install.packages("dartR");
#install.packages("tidyverse");
#install.packages("reshape2");

### Load packages

In [ ]:
# Load packages

library(vcfR);
library(Rtsne);
library(adegenet);
library(dartR);
library(tidyverse);
library(reshape2);

In [ ]:
# For reproductivity and visualisation in Jupyter

set.seed(0);
setwd("..");
options(repr.plot.width=16, repr.plot.height=8);

### Read data

In [ ]:
# Load genotypes fon VCF file

gt.vcf <- read.vcfR("mussels.snps.vcf.gz");
gt.vcf;

In [ ]:
# Load metadata

population <- read.delim("data/species.pop.tsv", header=FALSE, sep="\t", col.names=c("sample", "group", "colour", "loch", "age", "id", "larvaeYear"));
head(population);

## Fst

In [ ]:
gl.mussel <- vcfR2genlight(gt.vcf);
pop(gl.mussel) <- population$id;

In [ ]:
# Calculate Fst
gl.ft <- gl.fst.pop(gl.mussel, nboots = 1000, percent = 99, nclusters = 1, verbose = NULL);
gl.ft$Fsts;

In [ ]:
melted <- melt(gl.ft$Fsts, na.rm =TRUE);
melted_p <- melt(gl.ft$Pvalues, na.rm =TRUE);
melted <- cbind(melted, melted_p);
melted <- melted[c(1,2,3,6)];
melted$value.1 <- ifelse(melted$value.1>0.001,1,0);

ggplot(data = melted, aes(Var2, Var1, fill = value)) + theme_classic() + geom_tile(color = "white") + scale_fill_gradient(low = "white", high = "blue", name="Fst") + geom_point(aes(alpha=value.1) ,colour="black", stroke=NA) + scale_alpha_continuous(range=c(0, 1), guide="none") + labs( x = "Sampling Site", y = "Sampling Site") + theme(axis.text.x = element_text(angle = 45, vjust = 1, size = 11, hjust = 1), axis.text.y = element_text(size = 12), axis.ticks=element_blank(), axis.line=element_blank()) +  coord_fixed();

In [ ]:
# For publication only

#png("Figure_Fst.raw.png", 600, 300);
#ggplot(data = melted, aes(Var2, Var1, fill = value)) + theme_classic() + geom_tile(color = "white") + scale_fill_gradient(low = "white", high = "blue", name="Fst") + geom_point(aes(alpha=value.1) ,colour="black", stroke=NA) + scale_alpha_continuous(range=c(0, 1), guide="none") + labs( x = "Sampling Site", y = "Sampling Site") + theme(axis.text.x = element_text(angle = 45, vjust = 1, size = 11, hjust = 1), axis.text.y = element_text(size = 12), axis.ticks=element_blank(), axis.line=element_blank()) +  coord_fixed();
#dev.off();

## t-SNE analysis

In [ ]:
# Convert to matrix of genotypes

gt <- as.data.frame(t(extract.gt(gt.vcf, element = 'GT', as.numeric = FALSE)));
gt[gt == '0|0'] <- 0
gt[gt == '1|1'] <- 2
gt[gt == '0|1'] <- 1
gt[gt == '1|0'] <- 1
gt[is.na(gt)] <- 1; # replace NA by 1 (heterozygous)

gt[] <- lapply(gt, function(x) as.numeric(as.character(x)))
gt <- as.matrix(gt);

head(gt);

In [ ]:
# Run un-optimised t-SNE

tsne_out <- Rtsne(gt, initial_dims = 2, perplexity = 20, theta = 0.5, max_iter = 1000);

In [ ]:
plot(tsne_out$Y[, 1], tsne_out$Y[, 2], col=population$colour, cex=2, asp=1);

In [ ]:
# For publication only

#png("Figure_tSNE.raw.png", 600, 300);
#plot(tsne_out$Y[, 1], tsne_out$Y[, 2], col=population$colour, cex=2, asp=1);
#dev.off();

## Posterior membership probability (not used)

In [ ]:
# Convert to a genlight object

gl.mussel <- vcfR2genlight(gt.vcf);
ploidy(gl.mussel) <- 2;
pop(gl.mussel) <- population$id;

gl.mussel;

In [ ]:
# DAPC posterior probality assignment

gl.dapc <- dapc(gl.mussel, n.pca = 5, n.da = 2);

In [ ]:
# Prepare resulting matrix

dapc.results <- as.data.frame(gl.dapc$posterior);
dapc.results$pop <- pop(gl.mussel);
dapc.results$indNames <- rownames(dapc.results);

dapc.results <- pivot_longer(dapc.results, -c(pop, indNames));

dapc.results$colour <- population$colour[match(dapc.results$name, population$id)];
colnames(dapc.results) <- c("Original_Pop", "Sample", "Assigned_Pop", "Posterior_membership_probability", "Colour");
head(dapc.results);

col <- matrix(dapc.results$Colour);
rownames(col) <- dapc.results$Assigned_Pop;

In [ ]:
p <- ggplot(dapc.results, aes(x=Sample, y=Posterior_membership_probability, fill=Assigned_Pop));
p <- p + geom_bar(stat="identity");
p <- p + scale_fill_manual(values=col);
p <- p + facet_grid(~Original_Pop, scales = "free");
p <- p + theme(axis.title.x=element_blank(), axis.text.x=element_blank(), axis.ticks.x=element_blank());
p;


In [ ]:
# For publication only

#png("Figure_DAPC.raw.png", 600, 300);
#p;
#dev.off();